In [ ]:
#En este archivo planeo entrenar un modelo de 0 con 30 epocas aprox, teniendo en cuenta el overfitting y al final validando el dataset

In [ ]:
import torch
from torch import nn
from torchvision import datasets,transforms
from torch.utils.data import dataloader

path_training = "./training_set/"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from RedNeuronal.Model import  NeuronalNewtork #Para que encaje con MAIN
Model = NeuronalNewtork().to(device)
loss_fn = nn.CrossEntropyLoss()
print(device)

cuda


In [3]:
opti = torch.optim.Adam(params=Model.parameters(),lr=2e-4,betas=(0.9, 0.999),weight_decay=5e-4)

In [ ]:
png_to_tensor = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((.5,.5,.5),(.5,.5,.5)) #por el sobre entranamiento
])

data_set = datasets.ImageFolder(path_training,transform=png_to_tensor)
dl = dataloader.DataLoader(dataset=data_set,shuffle=True,num_workers=6,batch_size=64) #Estos parametros son especificos para mi PC, si tiene otro hardware se veten tener en cuenta

In [ ]:
#Training

size = len(dl.dataset)

rounds = 30

for z in range(rounds):
    correct = 0
    for batch, (x, y) in enumerate(dl):
        x,y = x.to(device),y.to(device)
        
        resultado = Model(x)
        loss = loss_fn(resultado,y)
        
        loss.backward()
        opti.step()
        opti.zero_grad()
        
        _, predicted = torch.max(resultado, 1)
        correct += (predicted == y).sum().item()

    print(f"Correctos : {100 * correct / size:.2f}%") 
    
    

Correctos : 63.09%
Correctos : 65.40%
Correctos : 67.26%
Correctos : 68.77%
Correctos : 69.90%
Correctos : 70.66%
Correctos : 71.90%
Correctos : 72.82%
Correctos : 73.37%
Correctos : 74.17%
Correctos : 74.51%
Correctos : 75.08%
Correctos : 75.56%
Correctos : 76.11%
Correctos : 76.82%
Correctos : 76.90%
Correctos : 77.55%
Correctos : 78.12%
Correctos : 78.35%
Correctos : 78.76%
Correctos : 79.20%
Correctos : 79.41%
Correctos : 79.63%
Correctos : 79.92%
Correctos : 80.48%
Correctos : 80.67%
Correctos : 80.57%
Correctos : 81.06%
Correctos : 81.44%
Correctos : 81.60%


In [ ]:
#Sanity

Model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x, y in dl:
        x, y = x.to(device), y.to(device)
        out = Model(x)
        _, preds = torch.max(out, 1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print(100 * correct / total)

86.0


In [8]:

total_params = sum(p.numel() for p in Model.parameters())
trainable_params = sum(p.numel() for p in Model.parameters() if p.requires_grad)

print("Total:", total_params)
print("Entrenables:", trainable_params)

Total: 1143690
Entrenables: 1143690


In [9]:
torch.save(Model.state_dict(), "./Modelos/my_modelo.pth")